<a href="https://colab.research.google.com/github/kmkurumi/colab/blob/main/%E3%80%8Ctacotron2_vits%E5%8D%95%E4%BA%BA_%E6%95%B0%E6%8D%AE%E9%9B%86%E5%A4%84%E7%90%86%E5%B7%A5%E5%85%B7%E9%9B%86_20230217_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 补充说明

1. demucs和slicer2可以自由选择，demucs默认放在前面以求slicer2分得更细；<br />如果因音频过长出错，可先执行slicer2后执行demucs

2. slicer2不是AI，是算法分割音频

3. 清除缓存会重置对应步骤产生的影响，但会删除对应步骤产生的数据，谨慎操作

4. 默认勾选`wv_show_log` 以便于排错，如果觉得输出的东西太影响视野可以勾掉，还你一个清爽的界面

5. 如果执行过程中出现一时解决不了的错误想要保留中间产物，或者demucs分离人声后想要保留背景音，<br />可以在导出时在 `wv_export_way` 填2，将过程中产生的所有文件打包保存到云盘

6. whisper转写有一定出错概率，能找到原台词一般还是原台词更好

## 准备

In [2]:
#@markdown 加载云端硬盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#@markdown 安装依赖

wv_show_log = True #@param :{type: "boolean"}

def do_install():
  wv_log_option = "" if wv_show_log else ">/dev/null 2>&1"
  !sudo apt install ffmpeg {wv_log_option}
  !sudo apt-get install p7zip-full p7zip-rar {wv_log_option}
  !pip install librosa soundfile {wv_log_option}
  !pip install demucs {wv_log_option}
  !pip install git+https://github.com/openai/whisper.git {wv_log_option}
  !git clone --recursive https://github.com/openvpi/audio-slicer ./slicer {wv_log_option}
  !pip install pydub {wv_log_option}
 
do_install()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-7build1).
The following NEW packages will be installed:
  p7zip-rar
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 44.8 kB of archives.
After this operation, 118 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 p7zip-rar amd64 16.02-3build1 [44.8 kB]
Fetched 44.8 kB in 1s (45.7 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debco

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-y8w8lsz7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-y8w8lsz7
  Resolved https://github.com/openai/whisper.git to commit 7858aa9c08d98f75575035ecd6481f462d66ca27
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230124-py3-none-any.whl size=1179424 sha256=ea5afded8f29566c18080df641a86040461787750974e0ad25418a4cd2aa8d84
  Stored in directory: /tmp/pip-ephem-wheel-cache-14p8t37u/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built open

In [4]:
#@markdown 定义常量和引入库
import os
import whisper
import librosa  # Optional. Use any library you like to read audio files.
import soundfile  # Optional. Use any library you like to write audio files.
from tqdm import tqdm
from slicer.slicer2 import Slicer

WV_RAW_DIR = "raws"  
WV_WAV_DIR = "wavs"
WV_OLD_WAV_DIR = "old_wavs"
WV_RAW_WAV_DIR = "raw_wavs"
WV_TMP_DIR = "tmp"
WV_FILELIST_DIR = "filelists" 
WV_FILELIST_FILE = "list.txt" 
WV_WHISPER_TMP_DIR = f"{WV_WAV_DIR}_before"

wv_whisper_model = 0

def wv_clean_empty_lines_for_one(filename):
    file_in = open(filename, "r", encoding="utf-8")
    content = file_in.readlines()
    file_in.close()
    file_out = open(filename, "w", encoding="utf-8")
    for i in range(len(content)):
        line = content[i]
        line = line.strip()
        if len(line) == 0:
            continue
        if i == len(content) - 1:
            print(line, file=file_out, end="")
        else:
            print(line, file=file_out)
    file_out.close()

def wv_clean_empty_lines_for_all():
  for _, _, files in os.walk(WV_FILELIST_DIR):
    for file in tqdm(files):
      wv_result_file = f'{WV_FILELIST_DIR}/{file}'
      wv_clean_empty_lines_for_one(wv_result_file)

In [5]:
#@markdown 清除上次解压缓存
wv_check_if = False #@param {type: "boolean"}
if wv_check_if:
  !rm -rf {WV_RAW_DIR}

In [6]:
#@markdown 解压
wv_show_log = True #@param :{type: "boolean"}
def fun_unzip():
  wv_log_option = "" if wv_show_log else ">/dev/null 2>&1"
  os.makedirs(WV_RAW_DIR, exist_ok=True)
  wv_raw_path = "/content/drive/MyDrive/dataset/YOUR_AUDIO_FILES.7z" #@param {type: "string"}
  !7z e {wv_raw_path} -o{WV_RAW_DIR} {wv_log_option}
fun_unzip()




7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/dataset/                                         
ERROR: No such file or directory
/content/drive/MyDrive/dataset/YOUR_AUDIO_FILES.7z



System ERROR:
Unknown error -2147024894


## 处理流程

In [ ]:
#@markdown 清除上次demucs缓存
wv_check_if = False #@param {type: "boolean"}
if wv_check_if:
  !rm -rf {WV_WAV_DIR} {WV_TMP_DIR} {WV_RAW_WAV_DIR}

In [ ]:
#@markdown ### 使用 demucs 去除背景杂音？（用时较长）
wv_use_demucs = True #@param {type: "boolean"}
# #@markdown 在过程结束（成功或失败）时删除临时文件？
# wv_use_demucs_delete_tmp = False #@param {type: "boolean"}
#@markdown 显示转换过程日志
wv_show_log = True #@param :{type: "boolean"}
#@markdown 选择模型
wv_demucs_model_name = "htdemucs_6s" #@param {type: "string"}

# (raws -> wavs)

# wavs  -> raw_wavs
#       -> wavs
def fun_use_demucs():
  if not os.path.exists(WV_WAV_DIR):
    !cp -r {WV_RAW_DIR} {WV_WAV_DIR}
  !mv {WV_WAV_DIR} {WV_RAW_WAV_DIR}
  !mkdir {WV_WAV_DIR}
  wv_log_option = "" if wv_show_log else ">/dev/null 2>&1"
  for _, _, files in os.walk(WV_RAW_WAV_DIR):
    files.sort()
    for file in tqdm(files):
      old_file = f'{WV_RAW_WAV_DIR}/{file}'
      new_file = f'{WV_WAV_DIR}/{".".join(file.split(".")[:-1])}.wav'
      filename = file.split('.')[0]
      !demucs --two-stems=vocals {old_file} -o {WV_TMP_DIR} -n {wv_demucs_model_name} {wv_log_option}
      !ffmpeg -i {WV_TMP_DIR}/{wv_demucs_model_name}/{filename}/vocals.wav  -acodec pcm_s16le -ac 1 -ar 22050 {new_file} {wv_log_option}
  # if wv_use_demucs_delete_tmp:
  #  !rm -rf {WV_TMP_DIR}

if wv_use_demucs:
  fun_use_demucs()

# 

关于demucs模型选择 （摘自官方[readme](https://github.com/facebookresearch/demucs/blob/main/README.md)）

The list of pre-trained models is:
- `htdemucs`: first version of Hybrid Transformer Demucs. Trained on MusDB + 800 songs. Default model.
- `htdemucs_ft`: fine-tuned version of `htdemucs`, separation will take 4 times more time
    but might be a bit better. Same training set as `htdemucs`.
- `htdemucs_6s`: 6 sources version of `htdemucs`, with `piano` and `guitar` being added as sources.
    Note that the `piano` source is not working great at the moment.
- `hdemucs_mmi`: Hybrid Demucs v3, retrained on MusDB + 800 songs.
- `mdx`: trained only on MusDB HQ, winning model on track A at the [MDX][mdx] challenge.
- `mdx_extra`: trained with extra training data (including MusDB test set), ranked 2nd on the track B
    of the [MDX][mdx] challenge.
- `mdx_q`, `mdx_extra_q`: quantized version of the previous models. Smaller download and storage
    but quality can be slightly worse.
- `SIG`: where `SIG` is a single model from the [model zoo](docs/training.md#model-zoo).

In [ ]:
#@markdown 清除上次slicer2缓存
wv_check_if = False #@param {type: "boolean"}
if wv_check_if:
  if os.path.exists(WV_OLD_WAV_DIR):
    !rm -rf {WV_WAV_DIR}
    !mv {WV_OLD_WAV_DIR} {WV_WAV_DIR}

In [ ]:
#@markdown ### 使用 [slicer2](https://github.com/openvpi/audio-slicer) 分割音频？

wv_use_slicer = True #@param {type: "boolean"}

#@markdown 下面的参数可以不改动
wv_slicer_threshold   = -40   #@param {type: "integer"}
wv_slicer_min_length  = 5000  #@param {type: "integer"}
wv_slicer_min_interval= 300   #@param {type: "integer"}
wv_slicer_hop_size    = 10    #@param {type: "integer"}
wv_slicer_max_sil_kept= 100   #@param {type: "integer"}

# wavs -> old_wavs
#      -> wavs
def fun_use_slicer():
  if not os.path.exists(WV_WAV_DIR):
    !cp -r {WV_RAW_DIR} {WV_WAV_DIR}
  !mv {WV_WAV_DIR} {WV_OLD_WAV_DIR}
  os.makedirs(WV_WAV_DIR, exist_ok=True)
  for _, _, files in os.walk(WV_OLD_WAV_DIR):
    files.sort()
    for file in tqdm(files):
      audio, sr = librosa.load(f'{WV_OLD_WAV_DIR}/{file}', sr=None, mono=False)  # Load an audio file with librosa.
      slicer = Slicer(
          sr=sr,
          threshold=wv_slicer_threshold,
          min_length=wv_slicer_min_length,
          min_interval=wv_slicer_min_interval,
          hop_size=wv_slicer_hop_size,
          max_sil_kept=wv_slicer_max_sil_kept
      )
      chunks = slicer.slice(audio)
      for i, chunk in enumerate(chunks):
          if len(chunk.shape) > 1:
              chunk = chunk.T  # Swap axes if the audio is stereo.
          filename = file.split('.')[0]
          soundfile.write(f'{WV_WAV_DIR}/{filename}_{str(i).zfill(4)}.wav', chunk, sr)  # Save sliced
if wv_use_slicer:
  fun_use_slicer()
else:
  !cp -r {WV_RAW_DIR} {WV_WAV_DIR}
      

In [ ]:
#@markdown 清除上次whisper缓存
wv_check_if = False #@param {type: "boolean"}
if wv_check_if:
  wv_whisper_model = 0
  if os.path.exists(WV_WHISPER_TMP_DIR):
    !rm -rf {WV_WAV_DIR}
    !mv {WV_WHISPER_TMP_DIR} {WV_WAV_DIR}
  !rm -rf {WV_FILELIST_DIR} /content/whisper-vits-japanese

In [ ]:
#@markdown ### whisper 转写 (用时较长)

#@markdown **单个音频不能超过30秒，可使用前面的slicer2分割**

#@markdown 0 - 不使用whisper转写

#@markdown 1 - 直接使用whisper转写

#@markdown 2 - 调用 [auto.py](https://github.com/AlexandaJerry/whisper-vits-japanese/blob/main/auto.py) 进一步分割音频
wv_whisper_option = 1 #@param {type: "integer"}
#@markdown whisper 语种
wv_language = "Japanese" #@param {type: "string"}
#@markdown whisper 模型
wv_whisper_model_name = "large" #@param {type: "string"}


关于whisper模型选择 （摘自官方[readme](https://github.com/openai/whisper/blob/main/README.md)）

**Available models and languages**

There are five model sizes, four with English-only versions, offering speed and accuracy tradeoffs. Below are the names of the available models and their approximate memory requirements and relative speed. 


|  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
|:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
|  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~1 GB     |      ~32x      |
|  base  |    74 M    |     `base.en`      |       `base`       |     ~1 GB     |      ~16x      |
| small  |   244 M    |     `small.en`     |      `small`       |     ~2 GB     |      ~6x       |
| medium |   769 M    |    `medium.en`     |      `medium`      |     ~5 GB     |      ~2x       |
| large  |   1550 M   |        N/A         |      `large`       |    ~10 GB     |       1x       |

In [ ]:
#@markdown 转写方式1 - 直接使用whisper转写

# snippet in https://github.com/openai/whisper/blob/main/README.md

# load model
def fun_use_whisper_way1_01():
  global wv_whisper_model
  if wv_whisper_model == 0:
    wv_whisper_model = whisper.load_model(wv_whisper_model_name)
# use model
def fun_use_whisper_way1_02():
  os.makedirs(WV_FILELIST_DIR, exist_ok=True)
  wv_result_file = f'{WV_FILELIST_DIR}/{WV_FILELIST_FILE}'
  wv_result = open(wv_result_file, "a", encoding="utf-8")

  for _, _, files in os.walk(WV_WAV_DIR):
    files.sort()
    for file in tqdm(files):
      audio = whisper.load_audio(f"{WV_WAV_DIR}/{file}")
      audio = whisper.pad_or_trim(audio)
      mel = whisper.log_mel_spectrogram(audio).to(wv_whisper_model.device)
      options = whisper.DecodingOptions(language=wv_language)
      result = whisper.decode(wv_whisper_model, mel, options)
      print(f'wavs/{file}|{result.text}', file=wv_result)

if wv_whisper_option == 1:
  fun_use_whisper_way1_01()
  fun_use_whisper_way1_02()
  wv_clean_empty_lines_for_all()

In [ ]:
#@markdown 转写方式2 - 调用 auto.py 进一步分割音频
# import logging
#@markdown Forked from: https://github.com/AlexandaJerry/whisper-vits-japanese/blob/main/Whisper_Vits_Japanese.ipynb
def wv_split_way2_01():
  # logging.setLevel(logging.INFO)

  print(f'1. dependency')
  wv_use_split_repo = "https://github.com/wind4000/whisper-vits-japanese" #@param {type: "string"}
  wv_use_split_branch = "fix-regex" #@param {type: "string"}
  !git clone {wv_use_split_repo} -b {wv_use_split_branch}

  print(f'2. prepare')
  wvj_audio_dir = f'whisper-vits-japanese/audio/'
  # !cp /content/whisper-vits-japanese/whisper/transcribe.py /usr/local/lib/python3.8/dist-packages/whisper
  # !cp /content/whisper-vits-japanese/whisper/utils.py /usr/local/lib/python3.8/dist-packages/whisper
  !rm -rf {wvj_audio_dir}
  !cp -r {WV_WAV_DIR} {wvj_audio_dir}

  print(f'3. whisper transcribe')
  for _, _, files in os.walk(wvj_audio_dir):
    files.sort()
    print(files)
    files = " ".join([wvj_audio_dir + file for file in files])
    !whisper {files} -o /content/whisper-vits-japanese/srt_files --language {wv_language} --model {wv_whisper_model_name}

def wv_split_way2_02():
  print(f'4. run whisper-vits-japanese/auto.py')
  %cd /content/whisper-vits-japanese
  !python auto.py
  %cd /content

  print(f'5. final')
  os.makedirs(WV_FILELIST_DIR, exist_ok=True)
  !cp /content/whisper-vits-japanese/filelists/train_filelist.txt {WV_FILELIST_DIR}/{WV_FILELIST_FILE}
  !sed -i "s;/content/whisper-vits-japanese/sliced_audio/;wavs/;" {WV_FILELIST_DIR}/{WV_FILELIST_FILE}
  !mv {WV_WAV_DIR} {WV_WAV_DIR}_before
  !mv /content/whisper-vits-japanese/sliced_audio {WV_WAV_DIR}

  wv_clean_empty_lines_for_all()
  
  print(f'\n6. done')

if wv_whisper_option == 2:
  %cd /content
  wv_split_way2_01()
  wv_split_way2_02()
  


## 导出

In [ ]:
#@markdown 导出方式

#@markdown 0 - 不导出

#@markdown 1 - 导出filelists和wavs

#@markdown 2 - 导出全部文件
wv_export_way = 1 #@param {type: "integer"}
wv_done_path = "/content/drive/MyDrive/dataset/YOUR_DATASET.7z" #@param {type: "string"}

def fun_export():
  if os.path.isfile(wv_done_path):
    print("指定的路径已存在文件，会修改或覆盖原文件，请更换文件名")
    return
  if wv_export_way == 1:
    !7z a {wv_done_path} {WV_FILELIST_DIR}/ {WV_WAV_DIR}/
    return
  if wv_export_way == 2:
    !7z a {wv_done_path} /content -xr!drive -xr!sample_data
    return
fun_export()
